# Document Scanner Project 
- scan a document 
- using open cv

In [1]:
#!pip install  imutils

In [1]:
# import the necessary packages

import numpy as np
import cv2
import imutils

In [2]:
# define order points function
def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left point of document
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect_coord = np.zeros((4, 2), dtype = "float32")

    # the top-left point will have the smallest sum, whereas 
    # the bottom-right point will have the largest sum
    summ = pts.sum(axis = 1)
    rect_coord[0] = pts[np.argmin(summ)]
    rect_coord[2] = pts[np.argmax(summ)]

    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect_coord[1] = pts[np.argmin(diff)]
    rect_coord[3] = pts[np.argmax(diff)]

    # return the ordered coordinates
    return rect_coord


In [3]:

def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually using order_point function
    rect_coord = order_points(pts)
    # tl - top left point
    # tr - top right point
    # br - bottom right 
    # bl- bottom left 
    
    (tl, tr, br, bl) = rect_coord

    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    #  or the top-right and top-left x-coordinates and y coordinate
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # x and y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a  top-down view of the image,
    #again specifying points in the top-left, top-right, bottom-right, and bottom-left order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")

    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect_coord, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    # return the warped image
    return warped

In [4]:
# store the path of image
image_path=r"C:\Users\shilp\OneDrive\Desktop\receipt.jpg"

In [5]:
# storing numpy array of image 
image = cv2.imread(image_path)

# ratio is height of image /the height of resized image (=500)
ratio = image.shape[0] / 500.0

In [6]:
image.shape

(3264, 2448, 3)

In [7]:
ratio

6.528

In [8]:
# saving the copy of image 
original = image.copy()

In [9]:
image = imutils.resize(image, height = 500)


In [10]:
image.shape

(500, 375, 3)

In [11]:
# convert the image to grayscale, 
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# smoothing the image using kernel size(5,5) ,i.e. blur the image
gray = cv2.GaussianBlur(gray, (5, 5), 0)
 
# finging edges using canny with minimum intensity gradient =75  
# and max intensity gradient =200
edged = cv2.Canny(gray, 75, 200)


In [12]:
# show the original image and the edge detected image
print("STEP 1: Edge Detection")
cv2.imshow("Image", image)
cv2.imshow("Edged", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()


STEP 1: Edge Detection


In [13]:
# find the contours in the edged image, keeping only the
# largest ones, and initialize the screen contour
cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]


In [14]:
# loop over the contours
for c in cnts:
    # approximate the contour
    perimeter = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * perimeter, True)
       
    # if our approximated contour has four points, then we
    # can assume that we have found our screen
    if (len(approx) == 4):
        screenCnt = approx
        break

In [15]:
 screenCnt

array([[[241, 142]],

       [[ 82, 151]],

       [[102, 414]],

       [[281, 388]]], dtype=int32)

In [16]:
# show the contour (outline) of the piece of paper
print("STEP 2: Find contours of paper")
cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
cv2.imshow("Outline", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


STEP 2: Find contours of paper


### Output scanned image

In [22]:
# apply the four point transform to obtain a top-down
# view of the original image
warped = four_point_transform(original, screenCnt.reshape(4, 2) * ratio)


# show the original and scanned images
print("STEP 3: Apply perspective transform")
cv2.imshow("Original", imutils.resize(original, height = 650))
cv2.imshow("Scanned", imutils.resize(warped, height = 650))
cv2.waitKey(0)

STEP 3: Apply perspective transform


-1

# Output scanned data from rectangular document


In [20]:
from skimage.filters import threshold_local
# convert the warped image to grayscale, then threshold it
#to give it that 'black and white' paper effect
warped1 = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
T = threshold_local(warped1, 11, offset = 10, method = "gaussian")
warped1 = (warped1 > T).astype("uint8") * 255

# show the original and scanned images
print("STEP 3: Apply perspective transform")
cv2.imshow("Original", imutils.resize(original, height = 650))
cv2.imshow("extract_data_Scanned", imutils.resize(warped1, height = 650))
cv2.waitKey(0)

STEP 3: Apply perspective transform


-1